In [1]:
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.signal as sigs
import numpy as np
import cmdstanpy ## import stan interface for Python
import os
import seaborn as sns
import pandas as pd
from importlib import reload 
import sys
sys.path.append("..")
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import arviz

import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore")
from statannotations.Annotator import Annotator
from itertools import product

tex_fonts = {
    # Use LaTeX to write all text
#     "text.usetex": True,
    "font.family": "Helvetica",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 7,
    "font.size": 7,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 6,
    "xtick.labelsize": 6,
    "ytick.labelsize": 6,
    "axes.grid": True,
    'grid.color': '#DDDDDD',
    'grid.linestyle': '-',
    'grid.linewidth': 0.3,
    "lines.markersize":5,
    "lines.markeredgewidth":1,
    'axes.axisbelow':True,
    'pdf.fonttype':42,
    'axes.linewidth':0.5,
        'xtick.major.width':0.5,
    'ytick.major.width':0.5,
    'ytick.minor.width':0.3,

    'ytick.major.pad':0.2,
        "xtick.major.size":3,
    "ytick.major.size":3,
}

plt.rcParams.update(tex_fonts)

if os.name == "nt": ## adds compiler to path in Windows
    cmdstanpy.utils.cxx_toolchain_path() 
    
%config InlineBackend.close_figures=False # keep figures open in pyplot
#%config InlineBackend.print_figure_kwargs = {'bbox_inches':"tight", 'pad_inches':2}

np.random.seed(2101)


ratio= 2/(1+np.sqrt(5))

width = 520.344

kwags = {"wspace":0.2}

my_pal = {"4cm": "g", 
          "4em": "m", 
          "donor":"#a4e0ef", 
          "host":"#ff7f0e",
         "WT":"#02feff",
         "Young":"#7d81fc",
         "Old":"#f50aff"}

my_marks = {"donor": "o", 
           "host": "X",
         "WT":"o",
         "Young CHIM":"P",
         "Old CHIM":"^"}

scatterkwags = {"palette": my_pal,
                  "markers":my_marks,
                  'edgecolor':"k",
}


def set_size(width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    #golden_ratio = (5**.5 - 1) / 2
    golden_ratio = 1

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

def enumerated_product(*args):
    yield from zip(product(*(range(len(x)) for x in args)), product(*args))

def myMAP(data):
    
#     counts, x = np.histogram(data,bins=(np.int(np.rint(np.sqrt(len(data))))))
#     x_ind = np.unravel_index(np.argmax(counts), counts.shape)
# #     arrayed = data.to_numpy()

# #     i_peaks, _ = sigs.find_peaks(arrayed)

# #     i_peaks
# #     arrayed[i_peaks]

# #     # Find the index from the maximum peak
# #     i_max_peak = i_peaks[np.argmax(_['widths'])]

# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]


# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]
#     return x[x_ind]

    arrayed = data.to_numpy()
    nparam_density = sts.kde.gaussian_kde(arrayed)
    x = np.linspace(np.min(arrayed), np.max(arrayed), 2000)
    nparam_density = nparam_density(x)

    return x[np.argsort(nparam_density)[-1]]

def myMAP_CI(datad):
    
#     counts, x = np.histogram(data,bins=(np.int(np.rint(np.sqrt(len(data))))))
#     x_ind = np.unravel_index(np.argmax(counts), counts.shape)
# #     arrayed = data.to_numpy()

# #     i_peaks, _ = sigs.find_peaks(arrayed)

# #     i_peaks
# #     arrayed[i_peaks]

# #     # Find the index from the maximum peak
# #     i_max_peak = i_peaks[np.argmax(_['widths'])]

# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]


# #     # Find the x value from that index
# #     x_max = arrayed[i_max_peak]
#     return x[x_ind]

    arrayedd = datad.to_numpy()
    nparam_densityd = sts.kde.gaussian_kde(arrayedd)
    xd = np.linspace(np.min(arrayedd), np.max(arrayedd), 2000)
    nparam_densityd = nparam_densityd(xd)

    lA_calcd, uA_calcd = np.percentile(datad, q=[5, 95])
    
    return [xd[np.argsort(nparam_densityd)[-1]],  lA_calcd, uA_calcd]

In [2]:
#splines all data >5
# run = "_r6"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_skin'+run

#splines remove >5 and <400
# run = "_r7"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_skin'+run

#simuluatenous remove >5
# run = "_r1"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run

#simuluatenous remove >5 and <400
# run = "_r2"
# location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run

#simuluatenous remove >5 tighter sigma prior
run = "_r3"
location = '/opt/mesh/tiree/elise/samples_1/cd4_all'+run


In [3]:

typei = "homog"

precursorc = ['4Naive','4EM','4CM','69']
populationc = ['4EM','4EM.CD69+']
ki67c = ['Kihi','Kimid','Kilo']
organc=['SK','LP']

# dfoldyfp = dfyfp[(dfyfp['organ'] == 'SK')]
# dfoldtom = dftom[(dftom['organ'] == 'SK')]

parnames = ["alpha_A","delta_A","beta","Source","efft","effy"] 
pretty_parnames = ["$\\alpha$_A","$\\delta$_A","$\\beta$","Source","$\\epsilon$_t","$\\epsilon$_y"] #


pardf = pd.DataFrame()

for idx, adhpop in enumerated_product(precursorc,populationc,ki67c,organc):
    fullloc = location+"/stan-cache-"+adhpop[1]+adhpop[0]+adhpop[2]+adhpop[3]+"_"+str(1)+"/"
    
    print(idx)
    print(adhpop)
    
    fl = []
    fb = []
    fh = []
    
    #data slicing and manulipulation

    sys.path.insert(1, fullloc)
    import paras
    reload(paras)
    
    for f_name in os.listdir(fullloc):
        if f_name.endswith('.csv')&f_name.startswith('branched'):
            print(f_name)
            fb.append(fullloc+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('linear'):
            print(f_name)
            fl.append(fullloc+f_name)
        elif f_name.endswith('.csv')&f_name.startswith('homog'):
            print(f_name)
            fh.append(fullloc+f_name)

    if typei == "branched":
        sam = cmdstanpy.from_csv(fb)
    else:
        sam = cmdstanpy.from_csv(fh)

        

    parests_flvl = [sam.stan_variable(pn) for pn in parnames]

    k_hat_fl =  sam.stan_variable("k_hat_y")
    kihi_fl =  sam.stan_variable("f_kihi_y_calc")
    
    if (adhpop[0]=='69')&(adhpop[1]=='4EM'):
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames)
        pardf_lvl['organ']=adhpop[3]
        pardf_lvl['population']=paras.populationc
        if adhpop[0]=='69':
            pardf_lvl['precursor']='CD'+paras.precursor+'-'
        else:
            pardf_lvl['precursor']=paras.precursor
        pardf_lvl['kisource']=adhpop[2]
        pardf_lvl['ones']=1
        pardf_lvl['alpha_adata']=1/paras.alpha_A
        pardf_lvl['delta_adata']=1/paras.delta_A
        pardf_lvl['betadata']=1/paras.beta
        pardf_lvl['efftdata']=paras.efft
        pardf_lvl['sourcedata']=paras.Source
        pardf_lvl['counts_t0']=paras.counts_t0_y
        pardf_lvl['TotalCount']=np.mean(np.sum(k_hat_fl[:,:,:],axis=2), axis=1)
        pardf_lvl['KihiFrac'] = np.mean(kihi_fl,axis=1)
        pardf_lvl["propprod"] = pardf_lvl["Source"]

        pardf = pd.concat([pardf,pardf_lvl])
    else:
        pardf_lvl=pd.DataFrame(np.transpose(parests_flvl), columns = parnames)
        pardf_lvl['organ']=adhpop[3]
        pardf_lvl['population']=paras.populationc
        if adhpop[0]=='69':
            pardf_lvl['precursor']='CD'+paras.precursor+'-'
        else:
            pardf_lvl['precursor']=paras.precursor
        pardf_lvl['kisource']=adhpop[2]
        pardf_lvl['ones']=1
        pardf_lvl['alpha_adata']=1/paras.alpha_A
        pardf_lvl['delta_adata']=1/paras.delta_A
        pardf_lvl['betadata']=1/paras.beta
        pardf_lvl['efftdata']=paras.efft
        pardf_lvl['sourcedata']=paras.Source
        pardf_lvl['counts_t0']=paras.counts_t0_y
        pardf_lvl['TotalCount']=np.mean(np.sum(k_hat_fl[:,:,:],axis=2), axis=1)
        pardf_lvl['KihiFrac'] = np.mean(kihi_fl,axis=1)
        pardf_lvl["propprod"] = 100*pardf_lvl["alpha_A"]*pardf_lvl["counts_t0"]/(pardf_lvl["alpha_A"]*pardf_lvl["counts_t0"]  +  pardf_lvl["Source"])


        pardf = pd.concat([pardf,pardf_lvl])
    #     pardf = pd.concat([pardf,pardf_bvl])



pardf["inv_alpha_A"] = pardf["ones"].div(pardf["alpha_A"].values)
pardf["inv_delta_A"] = pardf["ones"].div(pardf["delta_A"].values)
pardf["inv_beta"] = pardf["ones"].div(pardf["beta"].values)
pardf["frac_source"] = 100*pardf["Source"].div(pardf["counts_t0"].values)
pardf["lambda_A"] = (pardf["delta_A"] - pardf["alpha_A"])
pardf["ln_lambda"] = np.log(2)/(pardf["lambda_A"])



(0, 0, 0, 0)
('4Naive', '4EM', 'Kihi', 'SK')
homog-20240917163709_3.csv
homog-20240917163709_1.csv
homog-20240917163709_2.csv
homog-20240917163709_4.csv
homog-20240917163709_5.csv
(0, 0, 0, 1)
('4Naive', '4EM', 'Kihi', 'LP')
homog-20240917163708_1.csv
homog-20240917163708_2.csv
homog-20240917163708_4.csv
homog-20240917163708_3.csv
homog-20240917163708_5.csv
(0, 0, 1, 0)
('4Naive', '4EM', 'Kimid', 'SK')
homog-20240917163740_2.csv
homog-20240917163740_5.csv
homog-20240917163740_1.csv
homog-20240917163740_4.csv
homog-20240917163740_3.csv
(0, 0, 1, 1)
('4Naive', '4EM', 'Kimid', 'LP')
homog-20240917163733_5.csv
homog-20240917163733_1.csv
homog-20240917163733_2.csv
homog-20240917163733_4.csv
homog-20240917163733_3.csv
(0, 0, 2, 0)
('4Naive', '4EM', 'Kilo', 'SK')
homog-20240917163716_2.csv
homog-20240917163716_1.csv
homog-20240917163716_4.csv
homog-20240917163716_3.csv
homog-20240917163716_5.csv
(0, 0, 2, 1)
('4Naive', '4EM', 'Kilo', 'LP')
homog-20240917163727_1.csv
homog-20240917163727_5.csv

(3, 1, 2, 0)
('69', '4EM.CD69+', 'Kilo', 'SK')
homog-20240917164334_4.csv
homog-20240917164334_3.csv
homog-20240917164334_1.csv
homog-20240917164334_2.csv
homog-20240917164334_5.csv
(3, 1, 2, 1)
('69', '4EM.CD69+', 'Kilo', 'LP')
homog-20240917164343_4.csv
homog-20240917164343_3.csv
homog-20240917164343_1.csv
homog-20240917164343_5.csv
homog-20240917164343_2.csv


In [7]:

stats = pd.DataFrame()

populationc = ['4EM','4EM.CD69+']
organc=['SK','LP']

#which sets of fits to use
kimode = ["Kihi","Kihi","Kilo","Kimid"]
precursor = ["4EM",'CD69-',"4EM","4EM"]
m=0

for idx, adhpop in enumerated_product(organc, populationc):

    names_l = ["inv_alpha_A","inv_delta_A","ln_lambda","frac_source","inv_beta","propprod"]


    stats_lvl=pd.DataFrame(data=[myMAP_CI(pardf[(pardf['organ']==adhpop[0])&(pardf['population']==adhpop[1])&(pardf['kisource']==kimode[m])&(pardf['precursor']==precursor[m])][i]) for i in names_l], 
                           columns=['MAP','l_CI','u_CI'],index=names_l)
    
    stats_lvl['organ']=adhpop[0]
    stats_lvl['population']=adhpop[1]
    stats_lvl['kisource']=kimode[m]
    stats_lvl['precursor']=precursor[m]
    cols = stats_lvl.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]
    stats_lvl = stats_lvl[cols]
    stats_lvl = stats_lvl.reset_index()
    
    m=m+1
    stats = pd.concat([stats,stats_lvl])
    
    
stats = stats.sort_index()


stats.to_csv('parameters.csv')
